# Part 1

In [1]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))
import openvino

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


In [2]:
# import videoHtml
# videoHtml.videoHTML('Manufacturing', ['original_videos/Manufacturing.mp4'])

In [3]:
MODEL_PATH = '/data/models/intel/person-detection-retail-0013/FP16/person-detection-retail-0013'
VIDEO = '/data/resources/manufacturing.mp4'
QUEUE = '/data/queue_param/manufacturing.npy'
PEOPLE = 5


def submit(device, node):
    device_postfix = device
    if device.startswith('HETERO:FPGA'):
        device_postfix = 'FPGA'
        
    job_name = 'JOB_MANUFACTURING_' + device_postfix
    output = '/output/results/manufacturing/' + device_postfix
    params = '{} {} {} {} {} {}'.format(MODEL_PATH, device, VIDEO, QUEUE, output, PEOPLE)

    job_id = !qsub queue_job.sh -l nodes=1:{node} -d . -F "{params}" -N {job_name}
    return job_id

In [4]:
#Submit job to the queue
# cpu_job_id = submit('CPU', 'tank-870:i5-6500te')

# print(cpu_job_id[0])

cpu_job_id = !qsub queue_job.sh -d . -l "nodes=1:tank-870:i5-6500te" -F "/data/models/intel/person-detection-retail-0013/FP16/person-detection-retail-0013 CPU /data/resources/manufacturing.mp4 /data/queue_param/manufacturing.npy /output/results/manufacturing/ 2" -N store_core

import liveQStat
liveQStat.liveQStat()

import get_results
get_results.getResults(cpu_job_id[0], filename='output.tgz', blocking=True)

!tar zxf output.tgz
!cat stdout.log
!cat stderr.log

getResults() is blocking until results of the job (id:mHBOwLVSUGBoFdSYHl38N0BjpRKsyLda) are ready.
Please wait....Success!
output.tgz was downloaded in the same folder as this notebook.
Could not run Inference:  too many values to unpack (expected 2)
results/
results/manufacturing/
results/manufacturing/output_video.mp4
stderr.log
Traceback (most recent call last):
  File "person_detect.py", line 204, in main
    coords, image = pd.predict(frame)
  File "person_detect.py", line 113, in predict
    bounding_boxes, output_image = self.draw_outputs(bb_xy, image)
ValueError: too many values to unpack (expected 2)


# Part 2

In [ ]:
import matplotlib.pyplot as plt

device_list=['CPU', 'GPU', 'FPGA', 'MYRIAD']
inference_time=[]
fps=[]
model_load_time=[]

for device in device_list:
    with open('results/manufacturing/'+device+'/stats.txt', 'r') as f:
        inference_time.append(float(f.readline().split("\n")[0]))
        fps.append(float(f.readline().split("\n")[0]))
        model_load_time.append(float(f.readline().split("\n")[0]))

In [ ]:
plt.bar(device_list, inference_time)
plt.xlabel("Device Used")
plt.ylabel("Total Inference Time in Seconds")
plt.show()

In [ ]:
plt.bar(device_list, fps)
plt.xlabel("Device Used")
plt.ylabel("Frames per Second")
plt.show()

In [ ]:
plt.bar(device_list, model_load_time)
plt.xlabel("Device Used")
plt.ylabel("Model Loading Time in Seconds")
plt.show()

In [ ]:
print(f'| Device\t | Inference Time(s)\t | FPS\t\t | Load Time(s)\t |')
print(f'|----------------|-----------------------|---------------|---------------|')
for i,d in enumerate(device_list):
    print(f'| {device_list[i]}   \t | {inference_time[i]}\t\t\t | {fps[i]:.3f}\t | {model_load_time[i]:.3f}\t |')


In [ ]:
import numpy as np

fig = plt.figure()
p = fig.add_subplot(111)

width = 0.2
x = np.arange(len(device_list))
y1 = p.bar(x, inference_time, width=0.2, color='r', align='center')
y2 = p.bar(x + width, model_load_time, width=0.2, color='b', align='center')
# p.bar(x + width*2, fps, width=0.2, color='g', align='center')

p.set_ylabel('Seconds')
p.set_xticks(x + width/2)
p.set_xticklabels(device_list)
p.legend( (y1[0], y2[0]), ('Inference Time', 'Model Load Time') )
